In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%pip install geopy
%pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [4]:
# 데이터 로드
data = pd.read_csv("/content/drive/MyDrive/Project/hotels_with_coordinates.csv")
data

<ipython-input-4-05fb802ec9a8>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("/content/drive/MyDrive/Project/hotels_with_coordinates.csv")


,hotel,star,review,date,price,address,district,length,clean_review,latitude,longitude
0,유성 메이호텔,5,매우 만족합니다 깨끗하고 시설도좋고 주변에 먹을것도많고 주차자리도 매우 널하고 1층...,2024.10.18,60000,대전 유성구 봉명동 551-17,유성구,250,매우 만족합니다 깨끗하고 시설도 좋고 주변에 먹을 것도 많고 주차 자리도 매우 널하...,36.354776,127.342764
1,유성 메이호텔,4,주변에 숙소가 많은데 다른숙소를 주로 이용하다 이번에 처음 이용해보았는데 객실도 너...,2024.10.18,60000,대전 유성구 봉명동 551-17,유성구,262,주변에 숙소가 많은데 다른 숙소를 주로 이용하다 이번에 처음 이용해 보았는데 객실도...,36.354776,127.342764
2,유성 메이호텔,5,봉명동 술집에서 술마실때 위치가 너무 좋아요 항상 찾는곳으로 주차장도 지상도 있고 ...,2024.10.28,60000,대전 유성구 봉명동 551-17,유성구,255,봉명동 술집에서 술 마실 때 위치가 너무 좋아요 항상 찾는 곳으로 주차장도 지상도 ...,36.354776,127.342764
3,유성 메이호텔,5,짐 놓고갔엇는데 프론트 분 이 되게 친절하셨었어요 아 송주차장이 없지만 그래도 주변...,2024.10.29,60000,대전 유성구 봉명동 551-17,유성구,192,짐 놓고 갔었는데 프런트 분 이 되게 친절하셨었어요 아 송 주차장이 없지만 그래도 ...,36.354776,127.342764
4,유성 메이호텔,4,할머니 댁같은 느낌객실깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피는지 ...,2024.11.05,60000,대전 유성구 봉명동 551-17,유성구,259,할머니 댁 같은 느낌 객실 깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피...,36.354776,127.342764
...,...,...,...,...,...,...,...,...,...,...,...
233552,대전 유성 발렌타인,3,안마방 싸고 젛음,2017.10.20,"20,000",대전광역시 유성구 장대로 20-19,유성구,9,안마방 싸고 젛음,36.357106,127.337561
233553,대전 유성 발렌타인,5,합리적인가격에 편히 깔끔하게있다갑니다,2017.10.20,"20,000",대전광역시 유성구 장대로 20-19,유성구,20,합리적인 가격에 편히 깔끔하게 있다 갑니다,36.357106,127.337561
233554,대전 유성 발렌타인,4,50원 할인되서 되게싸게 갔어요가격대비 편안하고 좋았습니당,2017.10.18,"20,000",대전광역시 유성구 장대로 20-19,유성구,35,50원 할인돼서 되게 싸게 갔어요 가격 대비 편안하고 좋았습니다,36.357106,127.337561
233555,대전 유성 발렌타인,3,잘 머물다왔습니다,2017.10.18,"20,000",대전광역시 유성구 장대로 20-19,유성구,9,잘 머물다 왔습니다,36.357106,127.337561


In [5]:
# 비정상 데이터 제거
data = data[data['clean_review'].apply(lambda x: isinstance(x, str) and x.strip() != "")]

# 비정상적인 데이터 확인
problematic_rows = data[~data['clean_review'].apply(lambda x: isinstance(x, str) and x.strip() != "")]

print(f"문제 있는 데이터 개수: {len(problematic_rows)}")
print(problematic_rows)

문제 있는 데이터 개수: 0
Empty DataFrame
Columns: [hotel, star, review, date, price, address, district, length, clean_review, latitude, longitude]
Index: []


In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [7]:
# 모델과 토크나이저 불러오기
model = AutoModelForSequenceClassification.from_pretrained("/content/drive/MyDrive/Project/finetuned_model2")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Project/finetuned_model2")

In [8]:
model.eval()

ElectraForSequenceClassification(
  (electra): ElectraModel(
    (embeddings): ElectraEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=3)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): ElectraEncoder(
      (layer): ModuleList(
        (0-11): 12 x ElectraLayer(
          (attention): ElectraAttention(
            (self): ElectraSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): ElectraSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): L

In [9]:
model = model.to(device)

In [10]:
import torch
from tqdm import tqdm

def predict_sentiment_score_batch(review_texts, batch_size=64):
    all_scores = []
    total_batches = len(review_texts) // batch_size + (1 if len(review_texts) % batch_size > 0 else 0)  # 총 배치 수

    for i in tqdm(range(0, len(review_texts), batch_size), desc="Processing batches", total=total_batches):
        batch_texts = review_texts[i:i + batch_size]  # 현재 배치의 리뷰들

        try:
            # 배치 텍스트를 토크나이징하고 GPU로 이동
            inputs = tokenizer(batch_texts, padding=True, truncation=True, return_tensors="pt", max_length=512).to(device)

            with torch.no_grad():
                outputs = model(**inputs)  # 모델 예측

            predictions = torch.softmax(outputs.logits, dim=-1)  # logit --> softmax
            scores = predictions[:, 1].cpu().numpy()  # 긍정 클래스(1)에 대한 확률

            all_scores.extend(scores)  # 각 배치의 예측 확률

            # GPU 메모리 관리
            del inputs, outputs, predictions  # 불필요한 변수 삭제
            torch.cuda.empty_cache()

        except Exception as e:
            print(f"Error processing batch {i // batch_size + 1}: {e}")
            continue

    return all_scores

# 리뷰 데이터 예측
data['predicted_score'] = predict_sentiment_score_batch(data['clean_review'].tolist(), batch_size=64)
print(data[['clean_review', 'predicted_score']])

Processing batches: 100%|██████████| 3649/3649 [24:16<00:00,  2.51it/s]


                                             clean_review  predicted_score
0       매우 만족합니다 깨끗하고 시설도 좋고 주변에 먹을 것도 많고 주차 자리도 매우 널하...         0.984308
1       주변에 숙소가 많은데 다른 숙소를 주로 이용하다 이번에 처음 이용해 보았는데 객실도...         0.985063
2       봉명동 술집에서 술 마실 때 위치가 너무 좋아요 항상 찾는 곳으로 주차장도 지상도 ...         0.985439
3       짐 놓고 갔었는데 프런트 분 이 되게 친절하셨었어요 아 송 주차장이 없지만 그래도 ...         0.985395
4       할머니 댁 같은 느낌 객실 깔끔한데 오래된 느낌은 못 지우겠어요 전담을 옆방에서 피...         0.802685
...                                                   ...              ...
233552                                          안마방 싸고 젛음         0.982479
233553                            합리적인 가격에 편히 깔끔하게 있다 갑니다         0.983028
233554                50원 할인돼서 되게 싸게 갔어요 가격 대비 편안하고 좋았습니다         0.985431
233555                                         잘 머물다 왔습니다         0.977383
233556                                           깨끗 친절 좋음         0.984341

[233535 rows x 2 columns]


<ipython-input-10-4e0aafed61fb>:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['predicted_score'] = predict_sentiment_score_batch(data['clean_review'].tolist(), batch_size=64)


In [11]:
data.to_csv('predicted_sentiment_scores_v4.csv', index=False)